In [ ]:
# Moses Notebook v1

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

In [2]:
sim_av_patient_c34 = pd.read_csv("sim_av_patient_c34.csv", dtype=str)
sim_av_gene_c34    = pd.read_csv("sim_av_gene_c34.csv", dtype=str)

In [3]:
print("Patients:", sim_av_patient_c34.shape)
print("Genes:", sim_av_gene_c34.shape)

Patients: (157354, 12)
Genes: (100689, 29)


In [4]:
# Ensure GENDER = {1, 2}, map to human-readable sex
sim_av_patient_c34["GENDER"] = sim_av_patient_c34["GENDER"].astype(str).str.strip()
sex_map = {"1": "MALE", "2": "FEMALE"}
sim_av_patient_c34["SEX_LABEL"] = sim_av_patient_c34["GENDER"].map(sex_map).fillna("UNKNOWN")

# Clean ethnicity text
sim_av_patient_c34["ETHNICITY"] = (
    sim_av_patient_c34["ETHNICITY"].astype(str).str.strip().str.upper()
)

In [5]:
gene_calls = sim_av_gene_c34[["PATIENTID", "GENE_DESC"]].dropna()
gene_calls["PATIENTID"] = gene_calls["PATIENTID"].astype(str)

In [6]:
df_long = gene_calls.merge(
    sim_av_patient_c34[["PATIENTID", "SEX_LABEL", "ETHNICITY"]],
    on="PATIENTID", how="left"
)

In [7]:
df_wide = (
    df_long
    .pivot_table(index=["PATIENTID", "SEX_LABEL", "ETHNICITY"],
                 columns="GENE_DESC", values="GENE_DESC",
                 aggfunc="size", fill_value=0)
    .astype(int)
    .reset_index()
)

print("Shape after pivot:", df_wide.shape)

Shape after pivot: (43254, 105)


In [10]:
def fdr_bh(pvals, alpha=0.05):
    """Benjamini-Hochberg procedure for FDR control"""
    pvals = np.array(pvals)
    ranked_pvals = np.argsort(pvals)
    m = len(pvals)
    thresholds = (np.arange(1, m+1) / m) * alpha
    passed = np.zeros_like(pvals, dtype=bool)
    passed[ranked_pvals] = pvals[ranked_pvals] <= thresholds
    return passed

In [11]:
genes = [c for c in df_wide.columns if c not in ["PATIENTID", "SEX_LABEL", "ETHNICITY"]]

In [12]:
sex_results = []
for g in genes:
    tab = pd.crosstab(df_wide["SEX_LABEL"], df_wide[g])
    for col in [0, 1]:
        if col not in tab.columns:
            tab[col] = 0
    tab = tab[[0, 1]]
    chi2, p, _, _ = chi2_contingency(tab.values)

    prev = (tab[1] / tab.sum(axis=1)).to_dict()
    sex_results.append({
        "gene": g,
        "p_value": p,
        "prev_female": prev.get("FEMALE", np.nan),
        "prev_male": prev.get("MALE", np.nan),
        "female_minus_male": prev.get("FEMALE", np.nan) - prev.get("MALE", np.nan)
    })

In [13]:
sex_df = pd.DataFrame(sex_results).sort_values("p_value")
sex_df["significant_FDR_5%"] = fdr_bh(sex_df["p_value"])

In [14]:
eth_results = []
for g in genes:
    tab = pd.crosstab(df_wide["ETHNICITY"], df_wide[g])
    for col in [0, 1]:
        if col not in tab.columns:
            tab[col] = 0
    tab = tab[[0, 1]]
    chi2, p, _, _ = chi2_contingency(tab.values)

    prev_by_eth = (tab[1] / tab.sum(axis=1)).to_dict()
    row = {"gene": g, "p_value": p}
    row.update({f"prev_{k}": v for k, v in prev_by_eth.items()})
    eth_results.append(row)

In [15]:
eth_df = pd.DataFrame(eth_results).sort_values("p_value")
eth_df["significant_FDR_5%"] = fdr_bh(eth_df["p_value"])

In [16]:
print("\nTop 10 genes most associated with SEX (before FDR):")
print(sex_df.head(10)[["gene", "p_value", "prev_female", "prev_male", "significant_FDR_5%"]])


Top 10 genes most associated with SEX (before FDR):
             gene   p_value  prev_female  prev_male  significant_FDR_5%
85           PTEN  0.036006     0.003663   0.002503               False
74          NTRK2  0.067438     0.001221   0.001976               False
44          H3F3A  0.107619     0.000195   0.000000               False
73          NTRK1  0.204153     0.005571   0.004655               False
66            MYC  0.207404     0.000147   0.000395               False
19           CDH1  0.273171     0.000049   0.000219               False
48           IDH2  0.276642     0.000733   0.001097               False
18  CD274 (PD-L1)  0.344626     0.420568   0.415745               False
47           IDH1  0.350236     0.001514   0.001142               False
63           MSH6  0.376922     0.003468   0.002942               False


In [17]:
print("\nSignificant genes after FDR correction (SEX):")
print(sex_df[sex_df["significant_FDR_5%"]])


Significant genes after FDR correction (SEX):
Empty DataFrame
Columns: [gene, p_value, prev_female, prev_male, female_minus_male, significant_FDR_5%]
Index: []


In [18]:
print("\nTop 10 genes most associated with ETHNICITY (before FDR):")
print(eth_df.head(10)[["gene", "p_value", "significant_FDR_5%"]])


Top 10 genes most associated with ETHNICITY (before FDR):
         gene        p_value  significant_FDR_5%
66        MYC  8.966641e-164                True
91       SDHB   6.111055e-48                True
70       NPM1   4.550903e-36                True
6        BCL2   1.345809e-30                True
45   HIST1H3B   4.202908e-20                True
39       FLT3   3.236239e-16                True
11      BRCA1   3.391348e-16                True
84      POLE2   4.703604e-14                True
100       VHL   4.703604e-14                True
8        BCOR   2.735705e-08                True


In [19]:
print("\nSignificant genes after FDR correction (ETHNICITY):")
print(eth_df[eth_df["significant_FDR_5%"]])


Significant genes after FDR correction (ETHNICITY):
         gene        p_value  prev_0    prev_A    prev_B    prev_C    prev_D  \
66        MYC  8.966641e-164     0.0  0.000158  0.000000  0.000000  0.000000   
91       SDHB   6.111055e-48     0.0  0.000000  0.000000  0.000000  0.000000   
70       NPM1   4.550903e-36     0.0  0.000237  0.000000  0.000000  0.000000   
6        BCL2   1.345809e-30     0.0  0.000158  0.000000  0.000000  0.000000   
45   HIST1H3B   4.202908e-20     0.0  0.000026  0.000000  0.000000  0.000000   
39       FLT3   3.236239e-16     0.0  0.000210  0.000000  0.000767  0.027027   
11      BRCA1   3.391348e-16     0.0  0.000079  0.000000  0.000000  0.000000   
84      POLE2   4.703604e-14     0.0  0.000000  0.002469  0.000000  0.000000   
100       VHL   4.703604e-14     0.0  0.000000  0.002469  0.000000  0.000000   
8        BCOR   2.735705e-08     0.0  0.000026  0.000000  0.000000  0.000000   
97       TP53   4.134424e-07     0.0  0.001157  0.000000  0.000000 

In [20]:
sex_df.to_csv("gene_sex_associations.csv", index=False)
eth_df.to_csv("gene_ethnicity_associations.csv", index=False)